In [5]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from datasets import load_dataset, Dataset

#model_id = 'meta-llama/Llama-3.2-1B'
model_id = 'meta-llama/Llama-3.2-1B-Instruct'

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kuro_ds = load_dataset('Kurosawama/beam_search_DPO', split = 'train')

quant_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.bfloat16)
generation_config = GenerationConfig.from_pretrained(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = quant_config).to(dev)
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [6]:
kuro_ds

Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 340
})

In [29]:
def get_answer(dataset, ans_num):
    """
    dataset = huggingface dataset 
    ans_num = int ; Valor de la entrada del dataset que queremos analizar

    Nos regresa las respuestas del modelo filtradas. 
    """
    chosen = dataset['chosen'][ans_num][1]['content']
    rejected = dataset['rejected'][ans_num][1]['content']
    return chosen, rejected

cho1, rej1 = get_answer(kuro_ds, 0)
print(cho1, '\n', '---', '\n', rej1)

∀x (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ∀x (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ∀x (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina))  
 --- 
  Dependent(x) ::: x is a person dependent on caffeine.
             Drinks(x) ::: x regularly drinks coffee.
             Jokes(x) ::: x jokes about being addicted to caffeine.
             Unaware(x) ::: x is unaware that caffeine is a drug.
             Student(x) ::: x is a student.
             Premises:
             ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
             ∀x (Drinks(x) ⊕ Jokes(x)) ::: People who either regularly drink coffee or joke about being addicted to caffeine.
             ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
             (

In [30]:
cho1

'∀x (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ∀x (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ∀x (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina)) '

In [31]:
# '∀x|∃x'
cho1.split('∀x ')

['',
 '(DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ',
 '(DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ',
 '(¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina)) ']

In [34]:
import re
re.split('∀x|∃x|¬\(', cho1) # Casi pero no nos regesa la ER.

['',
 ' (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ',
 ' (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ',
 ' (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ',
 'Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ',
 'IsDependentOn(rina, caffeine) ⊕ Student(rina)) ']

**VER QUÉ VERGAS VAMOS A HACER PARA LA EVALUACIÓN Y LA SEPARACIÓN DE PREMISAS**